## Part 1: Preprocessing

In [26]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [27]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [28]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [29]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'DistanceFromHome', 'HourlyRate', 'TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole', 'PercentSalaryHike', 'NumCompaniesWorked','JobRole', 'JobLevel']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
print(X_df.dtypes)

"""Education	int64
Age	int64
DistanceFromHome	int64
JobSatisfaction	int64
OverTime	object
StockOptionLevel	int64
WorkLifeBalance	int64
YearsAtCompany	int64
YearsSinceLastPromotion	int64
NumCompaniesWorked	int64"""

Age                    int64
DistanceFromHome       int64
HourlyRate             int64
TotalWorkingYears      int64
YearsAtCompany         int64
YearsInCurrentRole     int64
PercentSalaryHike      int64
NumCompaniesWorked     int64
JobRole               object
JobLevel               int64
dtype: object


'Education\tint64\nAge\tint64\nDistanceFromHome\tint64\nJobSatisfaction\tint64\nOverTime\tobject\nStockOptionLevel\tint64\nWorkLifeBalance\tint64\nYearsAtCompany\tint64\nYearsSinceLastPromotion\tint64\nNumCompaniesWorked\tint64'

In [31]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)

In [32]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [33]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)

In [34]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
ohe_department = OneHotEncoder()

# Fit the encoder to the training data
ohe_department.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_train = ohe_department.transform(y_train[['Department']])
department_test = ohe_department.transform(y_test[['Department']])
department_train

<1102x3 sparse matrix of type '<class 'numpy.float64'>'
	with 1102 stored elements in Compressed Sparse Row format>

In [35]:
# Create a OneHotEncoder for the Attrition column
ohc_attrition = OneHotEncoder()        

# Fit the encoder to the training data
ohc_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train = ohc_attrition.transform(y_train[['Attrition']])
attrition_test = ohc_attrition.transform(y_test[['Attrition']])
attrition_train

<1102x2 sparse matrix of type '<class 'numpy.float64'>'
	with 1102 stored elements in Compressed Sparse Row format>

## Part 2: Create, Compile, and Train the Model

In [36]:
# Find the number of columns in the X training data.
X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(X_train.shape[1],))

# Create at least two shared layers
shared_layer1 = layers.Dense(units=64, activation='relu')
shared_layer2 = layers.Dense(units=128, activation='relu')

In [37]:
# Create a branch for Department
# with a hidden layer and an output layer
department_branch = shared_layer1(input_layer)
department_branch = shared_layer2(department_branch)
# Create the hidden layer
department_hidden = layers.Dense(units=32, activation='relu')(department_branch)


# Create the output layer
department_output = layers.Dense(units=3, activation='softmax', name='department_output')(department_hidden)


In [38]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_branch = shared_layer1(input_layer)
attrition_branch = shared_layer2(attrition_branch)
# Create the hidden layer
attrition_hidden = layers.Dense(units=32, activation='relu')(attrition_branch)
# Create the output layer
attrition_output = layers.Dense(units=2, activation='softmax', name='attrition_output')(attrition_hidden)


In [39]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'categorical_crossentropy'},
              metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      1,216 │ input_layer_1[0]… │
│                     │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │      8,320 │ dense_4[0][0],    │
│                     │                   │            │ dense_4[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 32)        │      4,128 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 32)        │      4,128 │ dense_5[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_6[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ dense_7[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,957 (70.14 KB)

 Trainable params: 17,957 (70.14 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
# TROUBLESHOOTING 
# CANT RUN THE MODEL - CHECKING SHAPES AND DATATYPES 
print(department_train_dense.shape)
print(attrition_train_dense.shape)
print (X_train_scaled.shape)
print(department_train_dense.dtype)
print(attrition_train_dense.dtype)
print(f"Department Data: {department_train_dense}")
print(f"Attrition Data: {attrition_train_dense}")
print(f"X_train_scaled: {X_train_scaled}")


(1102, 3)
(1102, 2)
(1102, 18)
float32
float32
Department Data: [[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
Attrition Data: [[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]
X_train_scaled: [[-0.40942386  0.87475435  0.12842688 ... -0.50934131  1.9471272
  -0.25036136]
 [-1.29889668 -0.50720476  0.9190773  ... -0.50934131 -0.51357713
  -0.25036136]
 [-0.07587155  0.49785641  1.46264946 ... -0.50934131  1.9471272
  -0.25036136]
 ...
 [-0.18705565 -0.63283741  0.42492079 ... -0.50934131 -0.51357713
  -0.25036136]
 [-1.41008078  1.87981553 -1.4034583  ... -0.50934131 -0.51357713
   3.9942266 ]
 [-0.63179206 -0.00467417 -1.00813309 ...  1.96332003 -0.51357713
  -0.25036136]]


In [42]:
# Convert sparse matrices to dense format
department_train_dense = department_train.toarray()
attrition_train_dense = attrition_train.toarray()

# Ensure the output data is in the correct format
y_train_dict = {
    'department_output': department_train_dense,
    'attrition_output': attrition_train_dense
}

# Train the model
model.fit(X_train_scaled,
          y_train_dict,
          epochs=100,
          shuffle=True,
          verbose=2)


Epoch 1/100


ValueError: Attr 'Toutput_types' of 'OptionalFromValue' Op passed list of length 0 less than minimum 1.

In [47]:
# Ensure the correct data types
department_train_dense = department_train_dense .astype('float32')
attrition_train_dense =attrition_train_dense .astype('float32')

# Prepare the dictionary for training
y_train_dict = {
    'department_output': department_train_dense,
    'attrition_output': attrition_train_dense
}

# Train the model
model.fit(
    X_train_scaled,
    y_train_dict,
    epochs=100,
    shuffle=True,
    verbose=2
)


Epoch 1/100


ValueError: Attr 'Toutput_types' of 'OptionalFromValue' Op passed list of length 0 less than minimum 1.

In [52]:
## TESTING DUMMY CELL - STILL DOESNT RUN
dummy_X = np.random.random((1102, 18)).astype('float32')
dummy_department_output = np.random.random((1102, 3)).astype('float32')
dummy_attrition_output = np.random.random((1102, 2)).astype('float32')

y_train_dict = {
    'department_output': dummy_department_output,
    'attrition_output': dummy_attrition_output
}

model.fit(
    dummy_X,
    y_train_dict,
    epochs=1,
    shuffle=True,
    verbose=2
)



ValueError: Attr 'Toutput_types' of 'OptionalFromValue' Op passed list of length 0 less than minimum 1.

In [55]:
# Evaluate the model with the testing data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_dict, verbose=2)

NameError: name 'X_test_scaled' is not defined

In [ ]:
# Print the accuracy for both department and attrition
print(f"Attrition Accuracy: {model_accuracy[1]}")
print(f"Department Accuracy: {model_accuracy[0]}")


Attrition predictions accuracy: 0.7880434989929199
Department predictions accuracy: 0.5


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE
-- ERROR -- I could not get this to compile to save my life. Above you see some trouble shooting cells ive added to check datatypes, i ahve updated/reinstalled tenserflow keras. im not sure if its something ive done or need to work with a tutor on
1. Unsure without final results
2. I used Softmax for the output layer instead of ReLU, this is because the Softmax will look at the 4 outputs, and the 4 values adding up to 1, "sort" the output based on the values
3. A few way this model might be improved the model would be hyper parameter tuning of the layers - learning rate, batch size, and number of nuerons or layers. Also watch for data overfitting, add dropout or look into early stopping to prevent overtraining. 